In [ ]:
%load_ext autoreload

In [ ]:
import GPy

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import ConstantKernel

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, OneHotEncoder

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import pickle

In [ ]:

from scipy.special import expit as invlogit, logit

In [ ]:
import theano.tensor as tt
import pymc3 as pm
import theano

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
sns.set_style("whitegrid")

In [ ]:
%autoreload
import mfgpc_opt as mfgpc
import utilities
import my_utils
from utilities_new import * 

In [ ]:
from sklearn.metrics import pairwise_distances

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
aliases = ['diabetes', 'german', 'satimage-1', 'mushroom', 'splice', 'spambase', 'hypothyroid', 'waveform-40']

In [ ]:
length_scale_bounds = (0.01, 10.0)
constant_value_bounds = (0.1, 10.0)

In [ ]:
def mcmc_predict(X0_l, y_l, X0_h, y_h, rho, f_scale_l, lengthscale_l, f_scale_d, lengthscale_d, noise=5*1e-4):
    f = np.hstack((y_l, y_h))
    n_l = len(X0_l)
    n_h = len(X0_h)
    cov_l = f_scale_l * pm.gp.cov.ExpQuad(X0_l.shape[1], lengthscale_l)
    cov_d = f_scale_d * pm.gp.cov.ExpQuad(X0_l.shape[1], lengthscale_d)

    
    K = tt.alloc(0.0, n_l + n_h, n_l + n_h)
    K = tt.set_subtensor(K[:n_l, :n_l], cov_l(X0_l))
    K = tt.set_subtensor(K[n_l:n_l + n_h, n_l:n_l + n_h], rho**2 * cov_l(X0_h) + cov_d(X0_h))
    K = tt.set_subtensor(K[:n_l, n_l:n_l + n_h], rho * cov_l(X0_l, X0_h))
    K = tt.set_subtensor(K[n_l:n_l + n_h, :n_l], rho * cov_l(X0_h, X0_l))
    K_noiseless = K.copy() + 1e-6 * tt.eye(n_l + n_h)
    K = tt.inc_subtensor(K[:n_l, :n_l], noise * tt.eye(n_l))
    K = tt.inc_subtensor(K[:, :], 1e-6 * tt.eye(n_l + n_h))

    K_s = tt.alloc(0.0, n_l + n_h, len(X))
    K_s = tt.set_subtensor(K_s[:n_l, :], rho * cov_l(X0_l, X))
    K_s = tt.set_subtensor(K_s[n_l:n_l + n_h, :], rho**2 * cov_l(X0_h, X) + cov_d(X0_h, X))

    _, trace = utilities.mcmc_mf_clf(len(X0_l), len(X0_h), K, K_noiseless, K_s, f, f_latent=None, trials=1000)
    trace_vals = trace.get_values('f_pred')


    return invlogit(np.mean(logit(trace_vals), axis = 0))

In [ ]:
hf_points = 75
c = 0.2
num_classes = 2
low_fidelity_factor = 3
length_scale_bounds = (0.01, 10.0)
constant_value_bounds = (0.1, 10.0)
df_id = 4

i = 6
#aliases = ['diabetes', 'german', 'satimage-1', 'mushroom', 'splice', 'spambase', 'hypothyroid', 'waveform-40']
aliases = ['diabetes', 'satimage-1', 'mushroom', 'waveform-40']

#print(aliases[i])
for df_id in aliases:
    X, y_gold = get_binary_dataset(df_id)
    X = StandardScaler().fit_transform(X)
    y = y_gold
    y_corrupted = (y + (np.random.rand(len(y)) < c).astype(int)) % 2

    alpha = hf_points/len(y)
    r_offset = [0]
    runs = 10
    r = 0
    is_y_train_good = False
    while not is_y_train_good:
        X_train, X_test, y_train, y_test, i_train, i_test = train_test_split_with_indices(X, y, test_size=1 - alpha, random_state=r + r_offset[0])
        is_y_train_good = (len(set(y_train)) == num_classes)
        if y_gold is not None:
            y_test = y_gold[i_test]
        if y_corrupted is not None:
            X_train_lf, _, y_train_lf, _ = train_test_split(X, y_corrupted, test_size=1 - alpha*low_fidelity_factor, random_state=r + r_offset[0] + 1)
            is_y_train_good &= (len(set(y_train_lf)) == num_classes)
        r_offset[0] += 1

    kernel = ConstantKernel(1, constant_value_bounds=constant_value_bounds) * RBF(0.1, length_scale_bounds=length_scale_bounds)
    mf_gpc = mfgpc.MultiFidelityGaussianProcessClassifier(kernel = kernel, rho = 0.0, n_restarts_optimizer = 10, eval_gradient=True)
    mf_gpc.fit(X_train_lf, y_train_lf, X_train, y_train)
    y_mfgpc_pred = mf_gpc.predict_proba(X)[:, 1]

    rho = mf_gpc.base_estimator_.rho
    f_scale_l, lengthscale_l = np.exp(mf_gpc.base_estimator_.kernel_l_.theta)
    f_scale_l = float(f_scale_l)
    f_scale_d, lengthscale_d = np.exp(mf_gpc.base_estimator_.kernel_d_.theta)
    f_scale_d = float(f_scale_d)

    y_mcmc_pred = mcmc_predict(X_train_lf, y_train_lf, X_train, y_train, 
                               rho, f_scale_l, lengthscale_l, f_scale_d, lengthscale_d)

    pp = PdfPages('figures/2019_mcmc_vs_lapl_'+str(df_id)+'_hf_'+str(hf_points)+'.pdf')

    plt.figure(figsize = (5, 5))
    plt.scatter(y_mcmc_pred, y_mfgpc_pred, s = 1)
    plt.axis([-0.05, 1.05, -0.05, 1.05])

    pp.savefig(bbox_inches='tight')
    pp.close()

    print('df_' + str(df_id))
    print('mfgpc laplace ROCAUC:', roc_auc_score(y, y_mfgpc_pred))
    print('mfgpc mcmc ROCAUC:', roc_auc_score(y, y_mcmc_pred))